In [49]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import KFold

boston=load_boston
X_data,t=load_boston(return_X_y=True) 
# split the data into training and validation set   
X_train, X_valid, t_train, t_valid = train_test_split(X_data,t,test_size=1/5, random_state=10) 

In [50]:
N = len(X_train)
M = len(X_valid)

def FSS(K, X_train, X_valid, t_train, t_valid):
    N = len(X_train)
    M = len(X_valid)

    S = np.ones((M,1))
    S_index = np.array([-1])
    # compute the distance between each valid and train points
    dist_temp = np.zeros((M,N))
    # sort distances for each validation point
    
    y = np.zeros((K,M))
    RMSE = np.zeros(K)

    for k in range (K):
        if (k in S_index)==False:   #跳过已经加入的特征
            X_train_temp=np.column_stack((S,X_train[:,k]))
            X_valid_temp=np.column_stack((S,X_valid[:,k]))  

        for i in range (M): # loop through each valid points
            for j in range (N): # loop through each train points
                diff=X_valid_temp[i]-X_train_temp[j]
                dist_temp[i,j]=np.dot(diff,diff) #squared euclidian distance
        ind_temp = np.argsort(dist_temp, axis=1)  # the kth smallest element on row i is dist[i,ind[i,k]]

        for i in range(M):  #遍历验证集，计算error
            for s in range(k):  # compute the avg
                y[k,i] += t_train[ind_temp[i,s]]    
            y[k,i] /= (k+1)
        z =  np.subtract(t_valid,y[k,:])
        err = np.dot(z,z)/M
        RMSE[k] = np.sqrt(err)  #将k时的error存储在RMSE里
    
    minIndex=np.argmin(RMSE)    # find the min index for RMSE
    S=np.column_stack((S,X_train[:,minIndex]))  # 添加该特征
    np.append(S_index,minIndex)
    ````


In [51]:
def kNN(K, X_train_one, X_valid_one, t_train, t_valid):

    N = len(X_train)
    M = len(X_valid)
    #compute distances from validation points to training points
    dist_one = np.zeros((M,N))

    for i in range(M):
        for j in range(N):
            z = (X_valid_one[i]-X_train_one[j])
            dist_one[i,j] = z*z
    #sort distances for each x_i
    ind_one = np.argsort(dist_one, axis=1) # dist(i,ind[k]) is the kth smallest distance to x_i
    #implement kNN with one feature

    y_one = np.zeros(M) # row k stores predictions made by (k+1)-NN
    err_one = np.zeros(K)

    for i in range(M):
        #compute prediction for x_i
        for s in range(K):
             y_one[i] += t_train[ind_one[i,s]]
        y_one[i] /= (K+1)
        # compute error for this k
    z = y_one- t_valid
    err_one= np.dot(z,z)/M
    RMSE_one = np.sqrt(err_one)

    return RMSE_one

In [52]:
def kFold(splits, X, t):
    kf=KFold(n_splits=splits)
    for train_index, test_index in kf.split(X):
        print(test_index)
        X_train, X_test = X[train_index], X[test_index]
        t_train, t_test = t[train_index], t[test_index]
        RMSE=RMSE+linearRegression(X_train,X_test)
    error =RMSE/splits

    return error
    

In [53]:
x=kFold(10,X_train,t_train)


[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40]
[41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64
 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81]
[ 82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99
 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117
 118 119 120 121 122]
[123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140
 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158
 159 160 161 162 163]
[164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181
 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199
 200 201 202 203]
[204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221
 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239
 240 241 242 243]
[244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261
 262 263 26

NameError: name 'RMSE' is not defined